In [1]:
import numpy as np
# Dados iniciais do problema
betaMin = 0.0001;
betaMax = 0.45;
nIter = 500;
fck = 30;
bw = 25;
h = 50;
cob = 2.5;
profLambda = 0.80;
alfaC = 0.85;
fiEstribo = 5.00;
fiLongTrac = 12.5;
fiLongComp = 5.00;
asTrac = 5*8;
asComp = 2*0.20;
modEAco = 21000;
fyk = 500;
defSt50 = 2.07/1000; #para E de 210 GPA
defSt60 = 2.48/1000;



In [2]:
# Força no concreto
fcd = fck/1.4/10;
print('fcd =',fcd)
fyd = fyk/1.15/10;
print('fyd =',fyd)
d = h - (cob+fiEstribo/10+fiLongTrac/2/10);
dlinha = cob+fiEstribo/10+fiLongComp/2/10;
#d = 17.65;
print('d =',d)

betaX2com3 = 0.26;
betaX3com4 = 0.45;

def calculonrd(betaX,betaX2com3,defSt50,modEAco,fyd,d,alfaC,fcd,bw,profLambda,asTrac):

    if betaX <= betaX2com3:
        defStTrac = 10/1000;
    else:
        defStTrac = (3.5/1000)/betaX - (3.5/1000)

    #print('defStTrac =',defStTrac)

    if defStTrac < defSt50:
        sigSt = -modEAco*defStTrac;
    else:
        sigSt = -fyd;
    
    
    #print('sigSt =',sigSt)    

    linhaNeutra = betaX*d
    
    defStComp = (defStTrac/(linhaNeutra-d))*(linhaNeutra-dlinha)
    
    if defStComp < defSt60:
        sigStComp = modEAco* defStComp;
    else:
        sigStComp = fyd;

    #print('LN =',linhaNeutra)  

    Rcc = (alfaC*fcd*bw)*(profLambda*linhaNeutra)
    #print('Rcc =',Rcc)  

    Rst = sigSt*asTrac
    #print('Rst =',Rst)  
    
    Rsc = sigStComp*asComp
    #print('Rst =',Rst)  

    nRd = Rcc + Rsc + Rst
    #print('Nrd =', nRd) 
    
    return Rcc,Rst,Rsc,nRd

for iteracao in range(100):
    

    print("iter =", iteracao)
    
    c = (betaMin + betaMax)/2
    print('Nova tentativa =',c)

    Fc,Fst,Fsc,nrdda = calculonrd(betaMin,betaX2com3,defSt50,modEAco,fyd,d,alfaC,fcd,bw,profLambda,asTrac)
    Fc,Fst,Fsc,nrddb = calculonrd(betaMax,betaX2com3,defSt50,modEAco,fyd,d,alfaC,fcd,bw,profLambda,asTrac)
    Fc,Fst,Fsc,nrddbisec = calculonrd(c,betaX2com3,defSt50,modEAco,fyd,d,alfaC,fcd,bw,profLambda,asTrac)
    print("f(a) =",nrdda)
    print("f(b) =",nrddb)
    print("f(c) =",nrddbisec)
    
    if (nrdda*nrddbisec) <= 0:
        betaMin = betaMin
        betaMax = c
        print('betaMin =',betaMin)
        print('betaMax =',betaMax)
    elif (nrdda*nrddbisec) > 0:
        betaMin = c
        betaMax = betaMax
        print('betaMin =',betaMin)
        print('betaMax =',betaMax)

betaX=betaMin
print('betaX =',betaX)
linhaNeutra = betaX*d
print('LN =',linhaNeutra)
Mrd = Fc*(d-0.50*profLambda*linhaNeutra)+Fsc*(d-dlinha)
print('Mrd =',Mrd)


fcd = 2.1428571428571432
fyd = 43.47826086956522
d = 46.375
iter = 0
Nova tentativa = 0.22505
f(a) = -1733.0825169317434
f(b) = -1003.7330684304072
f(c) = -1375.734329657255
betaMin = 0.22505
betaMax = 0.45
iter = 1
Nova tentativa = 0.337525
f(a) = -1375.734329657255
f(b) = -1003.7330684304072
f(c) = -1192.2197682802016
betaMin = 0.337525
betaMax = 0.45
iter = 2
Nova tentativa = 0.3937625
f(a) = -1192.2197682802016
f(b) = -1003.7330684304072
f(c) = -1098.0853730549195
betaMin = 0.3937625
betaMax = 0.45
iter = 3
Nova tentativa = 0.42188125
f(a) = -1098.0853730549195
f(b) = -1003.7330684304072
f(c) = -1050.9310129738824
betaMin = 0.42188125
betaMax = 0.45
iter = 4
Nova tentativa = 0.43594062499999997
f(a) = -1050.9310129738824
f(b) = -1003.7330684304072
f(c) = -1027.3369616494904
betaMin = 0.43594062499999997
betaMax = 0.45
iter = 5
Nova tentativa = 0.44297031249999996
f(a) = -1027.3369616494904
f(b) = -1003.7330684304072
f(c) = -1015.5361867073036
betaMin = 0.44297031249999996
betaMax =